In [1]:
"""
Predictors based on gammatone spectrograms

Assumes that ``make_gammatone.py`` has been run to create the high resolution
spectrograms.
"""
from pathlib import Path

In [2]:
import eelbrain

In [3]:
# Define paths to data, and destination for predictors
DATA_ROOT = Path("~").expanduser() / 'Data' / 'cocoha'
STIMULUS_DIR = DATA_ROOT / 'stimuli'
PREDICTOR_DIR = DATA_ROOT / 'predictors'

In [4]:
# load all the stimuli in the stimulus directory excluding .wav suffix
STIMULI_PATHS = [stimulus.stem for stimulus in STIMULUS_DIR.glob("*.wav")]

In [9]:
# ABOUT 8 MINUTES TO RUN

# If the directory for predictors does not exist yet, create it
PREDICTOR_DIR.mkdir(exist_ok=True)

# If the directory for predictors does exist skip the rest of the code, to avoid overwriting existing predictors
if any(PREDICTOR_DIR.iterdir()):
    print(f"Predictor directory {PREDICTOR_DIR} already contains files, skipping predictor generation to avoid overwriting existing predictors.")
    print("If you want to regenerate predictors, please delete the existing files in the predictor directory and run this code again.")
else:
    # Loop through stimuli
    for path in STIMULI_PATHS:
        # Load the high resolution gammatone spectrogram
        gt = eelbrain.load.unpickle(STIMULUS_DIR / f'{path}-gammatone.pickle')

        # Apply a log transform to approximate peripheral auditory processing
        gt_log = (gt + 1).log()
        # Apply the edge detector model to generate an acoustic onset spectrogram
        gt_on = eelbrain.edge_detector(gt_log, c=30)

        # Create and save 1 band versions of the two predictors (i.e., temporal envelope predictors)
        eelbrain.save.pickle(gt_log.sum('frequency'), PREDICTOR_DIR / f'{path}~gammatone-1.pickle')
        eelbrain.save.pickle(gt_on.sum('frequency'), PREDICTOR_DIR / f'{path}~gammatone-on-1.pickle')
        # Create and save 8 band versions of the two predictors (binning the frequency axis into 8 bands)
        x = gt_log.bin(nbins=8, func='sum', dim='frequency')
        eelbrain.save.pickle(x, PREDICTOR_DIR / f'{path}~gammatone-8.pickle')
        x = gt_on.bin(nbins=8, func='sum', dim='frequency')
        eelbrain.save.pickle(x, PREDICTOR_DIR / f'{path}~gammatone-on-8.pickle')

        # Create gammatone spectrograms with linear scale, only 8 bin versions
        x = gt.bin(nbins=8, func='sum', dim='frequency')
        eelbrain.save.pickle(x, PREDICTOR_DIR / f'{path}~gammatone-lin-8.pickle')
        # Powerlaw scale
        gt_pow = gt ** 0.6
        x = gt_pow.bin(nbins=8, func='sum', dim='frequency')
        eelbrain.save.pickle(x, PREDICTOR_DIR / f'{path}~gammatone-pow-8.pickle')

Predictor directory /Users/sylvestereley/Data/cocoha/predictors already contains files, skipping predictor generation to avoid overwriting existing predictors.
If you want to regenerate predictors, please delete the existing files in the predictor directory and run this code again.
